# 01 — Yield Curve Construction (Bills + Notes Bootstrap)

## Objective
Build a reusable **zero-coupon discount curve** from realistic market-quote structures:
- Treasury bills (money-market simple yields)
- Treasury notes (par yields, semiannual)

This curve is the risk-free foundation for:
- pricing fixed-coupon bonds
- scenario analysis (rate shocks)
- risk measures (DV01 / duration / convexity / KRD)

## Bootstrapping logic (high level)
We represent the curve by discount factors $D(t)$ at **knot maturities** and interpolate between knots.

### Bills (simple yield to discount factor)
For a bill with simple yield $y$ and year fraction $\tau$:
$$
D(T) = \frac{1}{1 + y \tau}.
$$

### Notes (par yield constraint)
A par note has price $1$ (per unit notional). With coupon rate $c$ and frequency $m$:
- coupon per period: $\frac{c}{m}$
- cashflow dates: $t_1,\dots,t_n=T$

Par condition:
$$
\sum_{i=1}^{n} \frac{c}{m} D(t_i) + D(T) = 1.
$$

When bootstrapping the next note maturity $T$, earlier $D(t_i)$ are known (from prior knots / interpolation). We solve a **1D root** for $D(T)$.

## Interpolation choice
We interpolate **log discount factors** linearly:
$$
\log D(t) \text{ is linear between knots.}
$$
This keeps discount factors positive and behaves smoothly for pricing.

## Quality controls implemented
- knot dates increasing
- $D(t)>0$
- monotone non-increasing DFs across maturity knots (typical positive-rate regime)
- short-end extrapolation between valuation date and first knot (flat cc zero implied by first knot)
- no long-end extrapolation beyond last knot

In [1]:
import numpy as np
import pandas as pd

from fixed_income_engine.curves import (
    ZeroCurve,
    bootstrap_curve_from_bills_notes,
    curve_qc_report,
)

val_date = pd.Timestamp("2026-02-13")

market = pd.DataFrame([
    {"type": "bill", "maturity": pd.Timestamp("2026-02-20"), "quote": 0.0525, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-03-13"), "quote": 0.0520, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-05-15"), "quote": 0.0515, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2026-08-14"), "quote": 0.0505, "day_count": "ACT/360"},
    {"type": "bill", "maturity": pd.Timestamp("2027-02-12"), "quote": 0.0485, "day_count": "ACT/360"},
    {"type": "note", "maturity": pd.Timestamp("2028-02-15"), "quote": 0.0450, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2031-02-15"), "quote": 0.0430, "coupon_freq": 2, "day_count": "30/360"},
    {"type": "note", "maturity": pd.Timestamp("2036-02-15"), "quote": 0.0425, "coupon_freq": 2, "day_count": "30/360"},
]).sort_values("maturity").reset_index(drop=True)

market

,type,maturity,quote,day_count,coupon_freq
0,bill,2026-02-20,0.0525,ACT/360,NaN
1,bill,2026-03-13,0.0520,ACT/360,NaN
2,bill,2026-05-15,0.0515,ACT/360,NaN
3,bill,2026-08-14,0.0505,ACT/360,NaN
4,bill,2027-02-12,0.0485,ACT/360,NaN
5,note,2028-02-15,0.0450,30/360,2.0
6,note,2031-02-15,0.0430,30/360,2.0
7,note,2036-02-15,0.0425,30/360,2.0


In [2]:
curve = bootstrap_curve_from_bills_notes(market, val_date, zero_day_count="ACT/365", enforce_monotone_df=True)

knot_df = pd.DataFrame({
    "knot_date": pd.to_datetime(curve.knot_dates),
    "df": np.exp(curve.knot_log_dfs),
})
knot_df

,knot_date,df
0,2026-02-20,0.998980
1,2026-03-13,0.995972
2,2026-05-15,0.987149
3,2026-08-14,0.975105
4,2027-02-12,0.953254
5,2028-02-15,0.893272
6,2031-02-15,0.790633
7,2036-02-15,0.642857


In [3]:
qc = curve_qc_report(curve)
qc

,date,tau,df,zero_cc,df_positive,df_monotone
0,2026-02-20,0.019178,0.998980,0.053202,True,True
1,2026-03-13,0.076712,0.995972,0.052616,True,True
2,2026-05-15,0.249315,0.987149,0.051878,True,True
3,2026-08-14,0.498630,0.975105,0.050559,True,True
4,2027-02-12,0.997260,0.953254,0.048006,True,True
5,2028-02-15,2.005479,0.893272,0.056278,True,True
6,2031-02-15,5.008219,0.790633,0.046907,True,True
7,2036-02-15,10.010959,0.642857,0.044135,True,True


## Interpretation
- The curve is stored as **knot discount factors** plus interpolation logic.
- Once bootstrapped, every downstream component (bond pricing, DV01, scenarios) depends on calling:
  - `curve.df(dates)` for discount factors
  - `curve.zero_rate_cc(dates)` for implied zero rates

With **one curve build**, we are able to perform **millions of instrument valuations**.